# 일별 분류별 신문게재기사 페이지에서 URL 불러오기

In [1]:
import requests
import datetime
from bs4 import BeautifulSoup as bs

먼저, URL을 조작해 과거 뉴스의 링크를 얻는 과정을 자동화하기 위해 다음 뉴스목록 URL의 구성요소를 분석했다.

<p style="text-align:center">https://news.daum.net/breakingnews/<span style="color:magenta">politics</span>?<span style="color:forestgreen">page=1</span>&<span style="color:dodgerblue">regDate=20200830</span></p>

제목과 사진만 있어서 본문 내용이 없는 기사나 라디오 인터뷰 형식으로 작성된 기사들을 제외하기 위해 출처가 종합지인 기사들만 추렸다.

In [2]:
paper_publishers = ['한국일보','문화일보','동아일보','서울신문','세계일보','경향신문','국민일보','중앙일보','한겨레','조선일보']

참고로, 선택할 수 있는 <span style="color:magenta">기사 분류</span>의 목록은 아래와 같다.

In [3]:
valid_categories = ('politics', 'society', 'economic', 'culture', 'entertain', 'digital', 'editorial')

그래서 만약 한 <span style="color:magenta">분류</span>가 주어졌을 때, 아래와 같은 함수를 통해 형식에 맞는 URL을 생성할 수 있다.

In [4]:
def make_pageurl(category, pagenum, date):
    """
    Function to return formatted URL given 3 input parameters
    """
    assert category in valid_categories, 'Invalid input category'
    assert len(str(date)) == 8, 'Invalid input date format'
    return f"https://news.daum.net/breakingnews/{category}?page={pagenum}&regDate={date}"

형식에 맞는 URL이 유효한 URL이 되려면 <span style="color:forestgreen">페이지 번호</span>와 <span style="color:dodgerblue">날짜</span>가 유효해야 한다. 날짜는 `datetime`패키지를 사용해 유효한 날짜들의 목록을 얻고, 유효한 페이지 번호들의 목록은 request를 전송해서 얻은 HTML파일에서 기사목록이 조회되지 않을 때까지 pagenum을 하나씩 증가시켜보면 얻을 수 있다.

## 유효한 날짜 목록 얻기

In [5]:
def date_formatting(date):
    """
    convert datetime.date object into yyyymmdd format string
    """
    year = str(date.year)
    month = str(date.month).rjust(2, '0')
    day = str(date.day).rjust(2, '0')
    return year + month + day

In [6]:
date_formatting(datetime.date(*[2020,9,2]))

'20200902'

In [7]:
def get_datelist(start, end):
    """
    Given start, end date in yyyymmdd format, obtain list of dates in between
    """
    assert len(str(start)) == 8, 'start date not in proper format: yyyymmdd'
    assert len(str(end)) == 8, 'end date not in proper format: yyyymmdd'
    
    start = str(start)
    end = str(end)
    
    start_list = [start[:4], start[4:6], start[6:]]
    end_list = [end[:4], end[4:6], end[6:]]
    
    start_date = datetime.date(*list(map(int, start_list)))
    end_date = datetime.date(*list(map(int, end_list)))
    delta = (end_date - start_date).days
    assert delta > 0, 'start date must precede end date'
    
    date_list = [start_date + datetime.timedelta(d) for d in range(delta+1)]
    return list(map(date_formatting, date_list))

In [8]:
get_datelist(20200830, 20200903)

['20200830', '20200831', '20200901', '20200902', '20200903']

## 유효한 페이지 번호 얻어서 유효한 URL들 목록 얻기

In [9]:
urllist = []
datelist = get_datelist(20200830, 20200901)

for date in datelist:
    pagenum = 1
    
    while True:
        response = requests.get(make_pageurl('politics', pagenum, date))
        parsed = bs(response.text, 'html.parser')
        body = parsed.find('div', attrs={'class':'box_etc'})

        if body.select('p.txt_none'):
            break
        else:
            articles = body.select('div.cont_thumb')
            for tag in articles:
                publisher = tag.find('span', attrs={'class':'info_news'}).get_text().split()[0]
                if publisher in paper_publishers:
                    urllist.append(tag.find('a', attrs={'class':'link_txt'})['href'])

            pagenum += 1

    print(f"URLs for articles of {date} scraped completely")

URLs for articles of 20200830 scraped completely
URLs for articles of 20200831 scraped completely
URLs for articles of 20200901 scraped completely


In [10]:
len(urllist)

1019